# Housing Prices in King County, WA: Support Vector Regression
Goal
- Use Support Vector Regression to create a model that predicts the sale price of homes given various attributes about the house

## Obtain Data

In [15]:
# global imports

# sklearn features
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

# pandas features
import pandas as pd

# supress scientific notation
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# import dataframes 
%store -r dfs

# assign dataframes to variables
X_train = dfs[0]
X_val = dfs[1]
X_test = dfs[2]
y_train = dfs[3]
y_val = dfs[4]
y_test = dfs[5] 

# check importing data frames worked
df = [X_train, X_val, X_test, y_train, y_val, y_test]
for d in df:
    print(d.shape)

(3181, 18)
(682, 18)
(682, 18)
(3181, 1)
(682, 1)
(682, 1)


## Train Support Vector Regression Model
Model Evaluation
- Use model to make predictions for price given test predictors
- Compute metrics to compare predictions with actual price for test dataset
    - Minimize mean absolute error and mean squared error
        - Mean absolute error: the average difference between the observed price and predicted price
        - Mean squared error: the average squared difference between observed price and predicted price
            - Gives a higher weight than mean absolute error for large errors

Steps to Train the Random Forest Model
1. Train a base model with default parameters
2. Evaluate base model using validation dataset
3. Train a model with preliminary best parameters chosen through random search
    - Test a wide range of parameter values
        - Choose the set of parameters that minimizes mean squared error
    - Random search is appropriate for preliminary estimates
        - Has a faster runtime than grid search
            - This is because it does not try all parameter values
4. Evaluate random search model using validation dataset
5. Train a model with best parameters chosen through grid search
    - Test a narrow range of parameter values
        - Choose the set of parameters that minimizes mean squared error
    - Grid search is appropriate for final estimates
        - Tries all combinations of parameters
6. Evaluate model using validation dataset

In [3]:
# general support vector regressor
svr_model = SVR()

# function that evaluates the model
def evaluate(model, test_pred, test_resp):
    # predictions
    predict = model.predict(test_pred)
    # metrics on test data
    test_mae = mean_absolute_error(test_resp,  predict)
    test_mse = mean_squared_error(test_resp,  predict)
    # print results
    print("Mean Absolute Error: %s" %test_mae)
    print("Mean Squared Error: %s" %test_mse)

In [4]:
# evaluate the base model
base_model = SVR(gamma = 'auto')
base_model.fit(X_train, y_train.values.ravel())
evaluate(base_model, X_val, y_val.values.ravel())

Mean Absolute Error: 210590.9825763046
Mean Squared Error: 101625618270.59404


In [5]:
# random search grid
random_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [1, 3, 5, 7],
    'gamma': ['auto', 'scale'],
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 1, 2]}

# fit random search
svr_random = RandomizedSearchCV(estimator = svr_model, param_distributions = random_grid, n_iter = 
                               100, cv = 3, scoring = 'neg_mean_squared_error', random_state = 123)
svr_random.fit(X_train, y_train.values.ravel())

# output best parameters from random search
svr_random.best_params_

{'kernel': 'linear', 'gamma': 'scale', 'epsilon': 0.01, 'degree': 1, 'C': 100}

In [6]:
# evaluate best random search model
best_random = svr_random.best_estimator_
evaluate(best_random, X_val, y_val.values.ravel())

Mean Absolute Error: 193774.889887809
Mean Squared Error: 91739663892.96799


In [7]:
# grid search parameters
params_grid = {
    'kernel': ['linear'],
    'gamma': ['scale'],
    'epsilon': [0.0075, 0.01, 0.0125],
    'degree': [1, 2],
    'C': [50, 75, 100, 125, 150]}

# fit grid search
svr_grid = GridSearchCV(estimator = svr_model, param_grid = params_grid, cv = 3, scoring = 
                       'neg_mean_squared_error')
svr_grid.fit(X_train, y_train.values.ravel())

# output best parameters from grid search
svr_grid.best_params_

{'C': 150,
 'degree': 1,
 'epsilon': 0.0075,
 'gamma': 'scale',
 'kernel': 'linear'}

In [8]:
# evaluate best grid search model
best_grid = svr_grid.best_estimator_
evaluate(best_grid, X_val, y_val.values.ravel())

Mean Absolute Error: 187840.9805491457
Mean Squared Error: 88153486487.66376


## Final Support Vector Regressor Model
- Train a model with best parameters found from grid search
- Evaluate the model using the test dataset
    - The model is not overfitted
        - The mean absolute error for training and test datasets are similar
        - The mean squared error for training and test datasets are similar

In [9]:
# check model performance on the test dataset
final_model = SVR(C = 150, degree = 1, epsilon = 0.0075, gamma = 'scale', kernel = 'linear')
final_model.fit(X_train, y_train.values.ravel())
evaluate(final_model, X_test, y_test.values.ravel())

Mean Absolute Error: 203642.65827867747
Mean Squared Error: 142434180920.8759


## Dataframe for Analysis
Steps to Create Dataframe for Test Data
1. Create dataframe svr_test_data with test rows by making a copy of y_test
2. Add columns 'predictions', 'difference', and 'abs_difference' to svr_test_data
3. Inner join data_pandas and svr_test_data
    - svr_test_data now has columns from data_pandas
4. Save the dataframe
    - Dataframe used in *WashingtonHouseSales-SupportVectorRegressionAnalysis* program

In [21]:
# create dataframe
svr_test_data = y_test
svr_test_data.head()

,price
2760,212000.00
1520,681500.00
594,255000.00
2083,636000.00
2564,400000.00


In [22]:
# create predictions dataframe
predictions = final_model.predict(X_test)
svr_test_data['predictions'] = predictions
svr_test_data['difference'] = svr_test_data['price'] - svr_test_data['predictions']
svr_test_data['abs_difference'] = svr_test_data['difference'].abs()
svr_test_data = svr_test_data[['predictions', 'difference', 'abs_difference']]
svr_test_data.head()

,predictions,difference,abs_difference
2760,476333.85,-264333.85,264333.85
1520,512049.35,169450.65,169450.65
594,437282.48,-182282.48,182282.48
2083,474491.77,161508.23,161508.23
2564,456107.74,-56107.74,56107.74


In [23]:
# import dataframe
%store -r data_pandas
data_pandas.head()

,price,bedroom,bathroom,sqftLot,floors,waterfront,view,condition,sqftAbove,sqftBelow,yrBuilt,yrWorked,location
0,313000.00,3.00,1.50,7912,1.50,0,0,3,1340,0,1955,2005,Seattle
1,260000.00,4.00,2.00,8625,1.00,0,0,4,1480,0,1974,1974,South Urban
2,469000.00,2.00,1.00,4400,1.00,0,0,3,1030,0,1924,2011,Seattle
3,1135000.00,4.00,2.75,8103,1.00,0,3,3,1970,1400,1970,2014,Seattle
4,409500.00,4.00,2.75,13000,1.00,0,0,3,1320,820,1968,1997,North


In [24]:
# merge dataframes
svr_test_data = svr_test_data.merge(data_pandas, how='inner', left_index=True, right_index=True)
svr_test_data = svr_test_data.reset_index(drop=True)
svr_test_data.head()

,predictions,difference,abs_difference,price,bedroom,bathroom,sqftLot,floors,waterfront,view,condition,sqftAbove,sqftBelow,yrBuilt,yrWorked,location
0,476333.85,-264333.85,264333.85,212000.00,2.00,1.50,1525,2.00,0,0,3,1020,0,2004,2004,Seattle
1,512049.35,169450.65,169450.65,681500.00,5.00,2.75,11700,1.00,0,0,3,1630,1630,1964,2000,East Urban
2,437282.48,-182282.48,182282.48,255000.00,3.00,1.75,9720,1.00,0,0,3,1050,500,1976,1976,South Rural
3,474491.77,161508.23,161508.23,636000.00,2.00,1.75,3600,1.50,0,0,5,1230,0,1925,1925,Seattle
4,456107.74,-56107.74,56107.74,400000.00,3.00,2.25,11266,2.00,0,0,3,2140,0,1986,1986,East Rural


In [25]:
# save dataframe 
%store svr_test_data

Stored 'svr_test_data' (DataFrame)
